## Load Data

Load's all years of the JSON boat data.
Indexes data into boat_index by boat's sail number and type and then stores data for every year.


In [1]:
from __future__ import print_function

import sys
import json
import codecs
import ast
from prettytable import PrettyTable
import pandas as pd

import demjson3
#from barely_json import parse

import re

#from parser.parser import parse_json, parse_json_glob
#from parser.util import log

# This is really ugly code that uses a lot of fragile regex's to convert the 
# JSON files (which aren't legal JSON) into something that Python will parse
def load_data(filename):
    s = codecs.open(filename, "r", "utf-8-sig").read()

    # replace ROCK''N''ROLL with ROCKNROLL
    s = re.sub(': \"([^"]*)\'\'([^\'\"]+)\'\'([^"]*)\"', ": \'\\1\\2\\3\'", s)
    # replace "foo's'blah" with "foosblah"
    s = re.sub(': \"([^"]*)\'([^\'\"]+)\'([^"]*)\"', ": \'\\1\\2\\3\'", s)
    # replace "foo's blah" with "foos blah"
    s = re.sub(': \"([^"]*)\'([^"]*)\"', ": \'\\1\\2\'", s)
    # replace 'foo "bar"' with 'foo bar'
    s = re.sub(': \'([^\'"]*)"([^"]*)"([^\']*)\'', ": \'\\1\\2\\3\'", s)

    s = re.sub(': None,', ': "None",', s)

    # replace ' with "
    s = re.sub('(?<!\\\\)\'', '\"', s)
    # replace , NN: with , "NN":
    s = re.sub(',\s+([0-9]+):', ", \"\\1\":", s)
    s = s.replace('(', '[')
    s = s.replace(')', ']')
    s = re.sub('\\\\', '', s)
    boats = json.loads(s)
    return boats

# This loads all of the JSON files into the "boats" dict
# The structure this is:
# boats[year] -> [ boat1, boat2, boat3, ... boatN ]
# The boat data is a direct translation from the JSON files
boats = {}
for i in range(2019,2024):
    filename = "ALL" + str(i) + ".json"
    print("loading from " + filename)
    boats[i] = load_data(filename)

# This converts the data into a boats_index dicts.
# The structure is:
# boats_index[<sailnu><boattype>]['years'][year] -> { <year-based data dict> }
# boats_index[<sailnu><boattype>]['keys'] -> { <global boat info data dict> }
# fields in the data dict are in the list below
# The goal of boat_index is to make it easy to see all of the data that we 
# care about indexed by the boat instead of by year.
boat_index = {}
for year in range(2019, 2024):
    for boat in boats[year]:
        key = (boat['sailnumber'], boat['boat']['type'])
        boat_data = {
            "name": boat['name'],
            "sail_number": boat['sailnumber'],
            "gph": boat['rating']['gph'], 
            "type": boat['boat']['type'], 
            "crew_weight": boat['boat']['sizes']['crew'],
            "spin_sqm": boat['boat']['sizes']['spinnaker'], 
            "asym_sqm": boat['boat']['sizes']['spinnaker_asym'],
            "8kt_run_angle": boat['vpp']['run_angle'][1],
            "8kt_run_vmg": boat['vpp']['run_vmg'][1],
            "10kt_run_angle": boat['vpp']['run_angle'][2],
            "10kt_run_vmg": boat['vpp']['run_vmg'][2],
            "12kt_run_angle": boat['vpp']['run_angle'][3],
            "12kt_run_vmg": boat['vpp']['run_vmg'][3],
            "14kt_run_angle": boat['vpp']['run_angle'][4],
            "14kt_run_vmg": boat['vpp']['run_vmg'][4],    
            "vpp": boat['vpp']
        }
        if not (key in boat_index):
            boat_index[key] = {} 
            boat_index[key]['years'] = {}
            boat_index[key]['keys'] = {}
        boat_index[key]['years'][year] = boat_data


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
loading from ALL2019.json
loading from ALL2020.json
loading from ALL2021.json
loading from ALL2022.json
loading from ALL2023.json


## Find boats that have changed spinnaker types

Look for boats that have had a symmetrical spinnaker only in one year and an asymmetrical spinnaker only in another year. Filter out any where the crew weight has changed. Store the output into interesting_boats.

In [2]:
def find_interesting_boats(boat_index):
    interesting_boats = []
    for boat_key in boat_index:
        boat = boat_index[boat_key]
        spin_only = False
        asym_only = False
        crew_min = 100000
        crew_max = 0
        for year in boat['years']:
            if boat['years'][year]['spin_sqm'] == 0 and boat['years'][year]['asym_sqm'] > 0:
                asym_only = True
                boat['keys']['asym'] = year
            if boat['years'][year]['spin_sqm'] > 0 and boat['years'][year]['asym_sqm'] == 0:
                spin_only = True
                boat['keys']['spin'] = year
            if boat['years'][year]['crew_weight'] < crew_min:
                crew_min = boat['years'][year]['crew_weight']
            if boat['years'][year]['crew_weight'] > crew_max:
                crew_max = boat['years'][year]['crew_weight']
            
        #print(boat)
        #print(spin_min, spin_max, asym_min, asym_max)
        #return
        #if (asym_min == 0 and asym_max > 0) or (spin_min == 0 and spin_max > 0):
        #    interesting_boats.append(boat)
        if spin_only and asym_only and (crew_min == crew_max):
            interesting_boats.append(boat)
    return interesting_boats
            
interesting_boats = find_interesting_boats(boat_index)
print(len(interesting_boats))



38


# Produce Table

Produce's a table that can be loaded in Excel for all of the candidate boats

In [3]:
def export_summary(interesting_boats):
    table = {
        "sail_num": [],
        "boat_type": [],
        "orc_years": [],
        "spin_sqm": [],
        "asym_sqm": [],
        "spin_vmg_8kts": [],
        "asym_vmg_8kts": [],
        "asym_advantage_8kts": [],
        "spin_vmg_10kts": [],
        "asym_vmg_10kts": [],
        "asym_advantage_10kts": [],
        "spin_vmg_12kts": [],
        "asym_vmg_12kts": [],
        "asym_advantage_12kts": [],
        "spin_vmg_14kts": [],
        "asym_vmg_14kts": [],
        "asym_advantage_14kts": []
    }
    for boat in interesting_boats:
        spin_vmg_8 = 0
        asym_vmg_8 = 0
        spin_vmg_10 = 0
        asym_vmg_10 = 0
        spin_vmg_12 = 0
        asym_vmg_12 = 0
        spin_vmg_14 = 0
        asym_vmg_14 = 0
        boat_sail_number = ""
        boat_name = ""
        boat_type = ""
        spin_sqm = 0
        asym_sqm = 0
        years = []
        for year in boat['years']:
            boat_year = boat['years'][year]
            boat_sail_number = boat_year['sail_number']
            boat_type = boat_year['type']
            boat_name = boat_year['name']
            sail_type = ""
            if not (boat_year['spin_sqm'] > 0 and boat_year['asym_sqm'] > 0):
                if boat_year['spin_sqm'] > 0:
                    spin_vmg_8 = boat_year['8kt_run_vmg']
                    spin_vmg_10 = boat_year['10kt_run_vmg']
                    spin_vmg_12 = boat_year['12kt_run_vmg']
                    spin_vmg_14 = boat_year['14kt_run_vmg']
                    spin_sqm = boat_year['spin_sqm']
                    sail_type = "s"
                if boat_year['asym_sqm'] > 0:
                    asym_vmg_8 = boat_year['8kt_run_vmg']
                    asym_vmg_10 = boat_year['10kt_run_vmg']
                    asym_vmg_12 = boat_year['12kt_run_vmg']
                    asym_vmg_14 = boat_year['14kt_run_vmg']
                    asym_sqm = boat_year['asym_sqm']
                    sail_type = "a"
                years.append(str(year) + ":" + sail_type)
        
        #if boat_sail_number == "NED/NED4937":
        #    print(boat)
        
        table["sail_num"].append(boat_sail_number)
        table["boat_type"].append(boat_type)
        table["orc_years"].append(years)
        table["spin_sqm"].append(spin_sqm)
        table["asym_sqm"].append(asym_sqm)
        table["spin_vmg_8kts"].append(spin_vmg_8)
        table["asym_vmg_8kts"].append(asym_vmg_8)
        table["asym_advantage_8kts"].append(asym_vmg_8 - spin_vmg_8)
        table["spin_vmg_10kts"].append(spin_vmg_10)
        table["asym_vmg_10kts"].append(asym_vmg_10)
        table["asym_advantage_10kts"].append(asym_vmg_10 - spin_vmg_10)
        table["spin_vmg_12kts"].append(spin_vmg_12)
        table["asym_vmg_12kts"].append(asym_vmg_12)
        table["asym_advantage_12kts"].append(asym_vmg_12 - spin_vmg_12)
        table["spin_vmg_14kts"].append(spin_vmg_14)
        table["asym_vmg_14kts"].append(asym_vmg_14)
        table["asym_advantage_14kts"].append(asym_vmg_14 - spin_vmg_14)
        
        
    df = pd.DataFrame(table)
    df.to_excel(r'export_dataframe.xlsx', index=False)
    # too many columns to print nicely in jupyter, but you can uncomment this for debugging
    #print(df)
    
export_summary(interesting_boats)

In [4]:
# find asym 36.7s
#
# 36.7s are an interesting boat to look at because there are a lot of them and many
# have been upgraded to asym kites.  This just finds the asym ones so I could poke
# around at the conversions
def find_asym_367(boats):
    table = {
        "sail_num": [],
        "name": [],
        "boat_type": [],
        "orc_year": [],
        "asym_sqm": [],
    }

    for year in boats:
        for boat in boats[year]:
            if "36.7" in boat['boat']['type'] and boat['boat']['sizes']['spinnaker'] == 0 and boat['boat']['sizes']['spinnaker_asym'] > 0:
                table["sail_num"].append(boat['sailnumber'])
                table["name"].append(boat['name'])
                table["boat_type"].append(boat['boat']['type'])
                table["orc_year"].append(str(year))
                table["asym_sqm"].append(boat['boat']['sizes']['spinnaker_asym'])
    
    df = pd.DataFrame(table)
    print(df.sort_values(by=["sail_num", "orc_year"]))
    #print(df)


    
find_asym_367(boats)

          sail_num                    name       boat_type orc_year  asym_sqm
14  ESP/ESP10930_C              THINK FAST      FIRST 36.7     2021     85.90
34  ESP/ESP10930_C              THINK FAST      FIRST 36.7     2023     85.90
12   ESP/ESP6269_C                  SAYULA      FIRST 36.7     2021     88.75
24   ESP/ESP6269_C                  SAYULA      FIRST 36.7     2022     88.75
1      ESP/ESP7544                   JAMBO      FIRST 36.7     2019     96.50
9      ESP/ESP7544  JAMBO MAR SIN PLASTICO      FIRST 36.7     2021     96.50
19     ESP/ESP7544  JAMBO MAR SIN PLASTICO      FIRST 36.7     2022     96.50
32    ESP/ITA555_C   O POETA E UN FINGIDOR      FIRST 36.7     2023     84.28
5       EST/EST379        AMSERV TOYOTA ST  First 36.7 mod     2021    121.84
15      EST/EST379        AMSERV TOYOTA ST  First 36.7 mod     2022    121.84
6     ITA/ITA14433              BLACK BULL      First 36.7     2021     93.19
8     ITA/ITA14889              FAAMU-SAMI      First 36.7     2

In [5]:
#
# This plots overlapping polars for all of the asym conversions that are
# listed in "find_interesting_boats"
#

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import copy
import plotly.io as pio
import math
pio.renderers.default='notebook_connected'

#
# the VPP uses boat speeds at all points of sail except for beat_vmg and run_vmg where
# it uses VMG.  This helper function converts VMG back to boat speed
#
def inverse_vmg(vmg, angle_degrees):
    angle = math.radians(angle_degrees)
    return vmg * 1 / math.cos(angle)
 
#
# This produces a polar plot for 1 or 2 VPPs from boat certificates
# 
# I use this to overlay a polar plot for sym and asym versions of the same boat to 
# be able to compare results
#
def plot_polar(name, tags, vpps):
    fig = go.Figure()
    colors = [
        ['darkviolet', 'darkgreen', 'darkorange', 'darkcyan', 'darkred', 'darkblue', 'black'],
        ['violet', 'lightgreen', 'yellow', 'cyan', 'red', 'blue', 'grey']]

    # we don't have enough colors defined to handle vpps for more than 2 boats
    assert(len(vpps) <= len(colors))
    for vpp_index in range(0, len(vpps)):
        vpp = vpps[vpp_index]
        for speed_index in range(0, len(vpp['speeds'])):
            beat_angle = vpp['beat_angle'][speed_index]
            run_angle = vpp['run_angle'][speed_index]
            angles = [ beat_angle ]
            angles.extend(vpp['angles'])
            insert_index = 0
            # insert the run_vmg at the proper point in the angles[] list.  
            # keep track of where we inserted to do the same for speeds
            # insert_index == 0 means append
            for angle_index in range(0, len(angles)):
                if run_angle < angles[angle_index]:
                    insert_index = angle_index
                    angles.insert(insert_index, run_angle)
            if (insert_index == 0):
                angles.append(run_angle)
            #print(angles)

            speeds = [ inverse_vmg(vpp['beat_vmg'][speed_index], beat_angle) ]
            for angle in vpp['angles']:
                speeds.append(vpp[str(angle)][speed_index])
            if insert_index == 0:
                speeds.append(inverse_vmg(vpp['run_vmg'][speed_index], 180 - run_angle))
            else:
                speeds.insert(insert_index, inverse_vmg(vpp['run_vmg'][speed_index], 180 - run_angle))
            #print(speeds)
            
            marker_sizes = [ 10 ]
            marker_sizes.extend([6] * (len(angles) - 2))
            if insert_index == 0:
                marker_sizes.append(10)
            else:
                marker_sizes.insert(insert_index, 10)
            
            marker_shapes = [ "diamond" ]
            marker_shapes.extend(["circle"] * (len(angles) - 2))
            if insert_index == 0:
                marker_shapes.append("diamond")
            else:
                marker_shapes.insert(insert_index, "diamond")

            fig.add_trace(
                go.Scatterpolar(
                    r = speeds,
                    theta = angles,
                    mode = 'lines+markers+text',
                    marker = dict(size = marker_sizes, symbol = marker_shapes),
                    line = dict(shape="spline"),
                    name = str(vpp['speeds'][speed_index]) + ":" + tags[vpp_index],
                    line_color = colors[vpp_index][speed_index]))
    fig.update_layout(
        title=name,
        showlegend=True,
        height=640,
        polar = dict(
            domain = dict(x = [1, 0], y = [0, 1]),
            sector = [-90, 90],
            angularaxis = dict(thetaunit = "degrees", dtick = 10, rotation=90, direction="clockwise")
        ))
    fig.show()
        
    return

def plot_interesting_boats(interesting_boats):
    for boat in interesting_boats:
        asym_year = boat['keys']['asym']
        spin_year = boat['keys']['spin']
        tags = ["asym:" + str(asym_year), "spin:" + str(spin_year)]
        vpps = [boat['years'][asym_year]['vpp'], boat['years'][spin_year]['vpp']]
        name = "type:" + boat['years'][asym_year]['type'] + ", sailno:" + boat['years'][asym_year]['sail_number'] + ", spin_sqm:" + str(boat['years'][spin_year]['spin_sqm']) +", asym_sqm:" + str(boat['years'][asym_year]['asym_sqm'])
        plot_polar(name, tags, vpps)

        
plot_interesting_boats(interesting_boats)
    
#b = copy.deepcopy(boats[2021][0])
#plot_polar(["2021", "2020"], [copy.deepcopy(boats[2021][0]), copy.deepcopy(boats[2020][0])])